# Training a Quantum Circuit to Predict CpG Site Methylation

This notebook demonstrates the process of training a Parameterized Quantum Circuit (PQC) to predict whether a CpG site in a reference genome will be methylated.

## Inputs

- Sequences of nucleotides surrounding a CpG site. Encoded as one of the set {A, C, T, G}
- Either 1 or 0 representing whether the majority of experiments found a site to be methylated or not, respectively

## Extracting Surrounding Sequences

This tool extracts sequences of the length provided from the reference genome. There is an option to extract only sequences with at least a certain number of experiments on the CpG sites.

In [1]:
from qenetics.qcpg import qcpg


## The Parameterized Quantum Circuit

### Encoding Circuit

Multi-Controlled X (MCX) gates are used to encode the nucledotide information at each location in the sequence. The active controls correspond to the binary representation of the index of each nucleotide and the four data qubits correspond to the nucleotides A, T, C, and G, respectively.

In [2]:
from numpy.typing import NDArray
import pennylane as qml

from qenetics.qcpg import qcpg
from qenetics.tools.cpg_sampler import string_to_nucleotides, Nucleodtide


sequence: list[Nucleodtide] = string_to_nucleotides("ATCGATCGATCGATCG")
data_register_size: int = 4
address_register_size: int = qcpg.calculate_address_register_size(
    len(sequence)
)
circuit_width: int = address_register_size + data_register_size

@qml.qnode(qml.device("default.qubit", wires=circuit_width))
def run_encode_circuit(
    sequence: list[Nucleodtide],
):
    qcpg.single_encode_all_nucleotides(sequence)
    return qml.expval(
        qml.PauliZ(wires=circuit_width - 2)
        @ qml.PauliZ(wires=circuit_width - 1)
    )

print(qml.draw(run_encode_circuit)(sequence=sequence))

0: ─╭○─╭○─╭○─╭○─╭○─╭○─╭○─╭○─╭●─╭●─╭●─╭●─╭●─╭●─╭●─╭●─┤       
1: ─├○─├○─├○─├○─├●─├●─├●─├●─├○─├○─├○─├○─├●─├●─├●─├●─┤       
2: ─├○─├○─├●─├●─├○─├○─├●─├●─├○─├○─├●─├●─├○─├○─├●─├●─┤       
3: ─├○─├●─├○─├●─├○─├●─├○─├●─├○─├●─├○─├●─├○─├●─├○─├●─┤       
4: ─╰X─│──│──│──╰X─│──│──│──╰X─│──│──│──╰X─│──│──│──┤       
5: ────╰X─│──│─────╰X─│──│─────╰X─│──│─────╰X─│──│──┤       
6: ───────╰X─│────────╰X─│────────╰X─│────────╰X─│──┤ ╭<Z@Z>
7: ──────────╰X──────────╰X──────────╰X──────────╰X─┤ ╰<Z@Z>


### Ansatz

Below is the code to draw the second half of the circuit, which applies parameterized gates to the existing encoded state in an attempt to train a model that correctly categorizes each sequence as methylated or not.

In [3]:
from pennylane import numpy as pnp


@qml.qnode(qml.device("default.qubit", wires=circuit_width))
def run_ansatz_circuit(
    parameters: NDArray[float],
):
    layer_quantity: int = parameters.shape[0]
    wire_quantity: int = parameters.shape[1]
    rotations_quantity: int = parameters.shape[2]
    for layer_index in range(layer_quantity):
        qml.StronglyEntanglingLayers(
            parameters[layer_index, :, :].reshape(
                (1, wire_quantity, rotations_quantity)
            ),
            range(wire_quantity),
        )
    return qml.expval(
        qml.PauliZ(wires=circuit_width - 2)
        @ qml.PauliZ(wires=circuit_width - 1)
    )

unique_nucleotide_quantity: int = 4
layer_quantity: int = 2
params_shape = qml.StronglyEntanglingLayers.shape(
        n_layers=layer_quantity,
        n_wires=address_register_size + unique_nucleotide_quantity,
    )
parameters: NDArray = pnp.random.default_rng().random(size=params_shape)
print(qml.draw(run_ansatz_circuit)(parameters=parameters))

0: ─╭StronglyEntanglingLayers(M0)─╭StronglyEntanglingLayers(M1)─┤       
1: ─├StronglyEntanglingLayers(M0)─├StronglyEntanglingLayers(M1)─┤       
2: ─├StronglyEntanglingLayers(M0)─├StronglyEntanglingLayers(M1)─┤       
3: ─├StronglyEntanglingLayers(M0)─├StronglyEntanglingLayers(M1)─┤       
4: ─├StronglyEntanglingLayers(M0)─├StronglyEntanglingLayers(M1)─┤       
5: ─├StronglyEntanglingLayers(M0)─├StronglyEntanglingLayers(M1)─┤       
6: ─├StronglyEntanglingLayers(M0)─├StronglyEntanglingLayers(M1)─┤ ╭<Z@Z>
7: ─╰StronglyEntanglingLayers(M0)─╰StronglyEntanglingLayers(M1)─┤ ╰<Z@Z>

M0 = 
[[[0.96361589 0.79293482 0.23048376]
  [0.23390801 0.82517604 0.31622458]
  [0.71361937 0.24813393 0.50982154]
  [0.76518519 0.53301164 0.27281518]
  [0.08983888 0.16238679 0.06364833]
  [0.30117034 0.15672925 0.0931274 ]
  [0.37797982 0.21020801 0.05751823]
  [0.74530049 0.46723167 0.59926759]]]
M1 = 
[[[0.94285808 0.62429347 0.9601778 ]
  [0.63695646 0.71703647 0.91392788]
  [0.96942012 0.44169122 0.0154

The [StronglyEntanglingLayers](https://docs.pennylane.ai/en/stable/code/api/pennylane.StronglyEntanglingLayers.html) ansatz provides a maximally entangled quantum circuit with maximal degrees of freedom in the rotation gates. Below is a graphic representation of the ansatz. 

![Strongly Entangling Layers](images/strongly_entangled.png "The StronglyEntanglingLayers circuit from Pennylane")

## Training the Model

The sequences are read in and first split into a training and validation set. K-fold cross-validation is then performed on the training set by using non-overlapping training and test sets from the larger training set.

In [4]:
from pathlib import Path
from tempfile import TemporaryDirectory

from sklearn.model_selection import train_test_split, KFold

from qenetics.qcpg import qcpg
from qenetics.tools import cpg_sampler


seed: int = 50
sequences_filepath = Path("data/sequences.csv")
sequences, methylations = cpg_sampler.load_methylation_samples(sequences_filepath)
print(f"Total samples: {len(sequences)}")

(
        performance_sequences,
        validation_sequences,
        performance_methylation,
        validation_methylation,
    ) = train_test_split(sequences, methylations, random_state=seed, train_size=0.8, test_size=0.2)
print(f"Number of validation samples: {len(validation_sequences)}")

unique_nucleotide_quantity: int = 4
fold_quantity: int = 4
layer_quantity: int = 2
iteration_quantity: int = 5
for fold_number, (train_index, test_index) in enumerate(KFold(
    n_splits=fold_quantity, random_state=seed, shuffle=True).split(performance_sequences)):
    print(f"Training fold {fold_number} with {len(train_index)} training samples "
          f"and {len(test_index)} test samples.")
    training_sequences: list[list[Nucleodtide]] = [performance_sequences[index] for index in train_index]
    training_methylation: list[int] = [performance_methylation[index] for index in train_index]
    test_sequences: list[list[Nucleodtide]] = [performance_sequences[index] for index in test_index]
    test_methylation: list[int] = [performance_methylation[index] for index in test_index]
    address_register_size: int = qcpg.calculate_address_register_size(
        len(training_sequences[0])
    )
    params_shape = qml.StronglyEntanglingLayers.shape(
        n_layers=layer_quantity,
        n_wires=address_register_size + unique_nucleotide_quantity,
    )
    parameters: NDArray = pnp.random.default_rng().random(size=params_shape)
    with TemporaryDirectory() as temp_dir:
        temporary_metrics_file = Path(temp_dir) / "temp_metrics.csv"
        trained_parameters, loss_history, metrics_history = (
            qcpg.train_strongly_entangled_qcpg_circuit(
                parameters,
                training_sequences,
                training_methylation,
                test_sequences,
                test_methylation,
                temporary_metrics_file,
                iteration_quantity,
            )
        )
        print(f"\tLoss: {loss_history[-1]}")
        print(f"\tPerformance metrics: {metrics_history[-1]}")
    

Total samples: 11
Number of validation samples: 3
Training fold 0 with 6 training samples and 2 test samples.


c:\Users\chris\AppData\Local\Programs\Python\Python313\Lib\site-packages\pennylane\ops\op_math\composite.py:211: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


	Loss: 1.0349516868591309
	Performance metrics: Metrics(confusion_matrix=ConfusionMatrix(true_positives=0, false_positives=1, false_negatives=0, true_negatives=1), accuracy=0.5, true_positive_rate=0, false_positive_rate=0.5, f1=0.0)
Training fold 1 with 6 training samples and 2 test samples.
	Loss: 1.0362305641174316
	Performance metrics: Metrics(confusion_matrix=ConfusionMatrix(true_positives=0, false_positives=0, false_negatives=0, true_negatives=2), accuracy=1.0, true_positive_rate=0, false_positive_rate=1.0, f1=0)
Training fold 2 with 6 training samples and 2 test samples.
	Loss: 0.8948051333427429
	Performance metrics: Metrics(confusion_matrix=ConfusionMatrix(true_positives=1, false_positives=1, false_negatives=0, true_negatives=0), accuracy=0.5, true_positive_rate=1.0, false_positive_rate=0.0, f1=0.6666666666666666)
Training fold 3 with 6 training samples and 2 test samples.
	Loss: 0.8348903059959412
	Performance metrics: Metrics(confusion_matrix=ConfusionMatrix(true_positives=0,